In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 5-4. 演算子オーバーロード

### 5-4-2. 実例

#### コード5-24.	Point2D.jl（加算・減算の定義を含む）

In [2]:
abstract type AbstractPoint{T<:Real} end

struct Point2D{T} <: AbstractPoint{T}
    x::T
    y::T
end

function Base.:+(p1::Point2D, p2::Point2D)
    Point2D(p1.x + p2.x, p1.y + p2.y)
end

Base.:-(p1::Point2D, p2::Point2D) = Point2D(p1.x - p2.x, p1.y - p2.y)

#### コード5-25. `Point2D` の動作例（`+`/`-` 演算子による加減算）

In [3]:
p1 = Point2D(1.0, 2.0)

Point2D{Float64}(1.0, 2.0)

In [4]:
p2 = Point2D(3.0, 4.0)

Point2D{Float64}(3.0, 4.0)

In [5]:
p1 + p2

Point2D{Float64}(4.0, 6.0)

In [6]:
p1 - p2

Point2D{Float64}(-2.0, -2.0)

#### 仮想コード5-1.	`+` 演算子を誤って多重定義してしまった際のエラー例

```julia
julia> p1 + p2
ERROR: MethodError: no method matching +(::Float64, ::Float64)
You may have intended to import Base.:+
Stacktrace:
 [1] +(p1::Point2D{Float64}, p2::Point2D{Float64})
   @ Main ./REPL[XX]:2
 [2] top-level scope
   @ REPL[YY]:1
```

#### 仮想コード5-2.	`+` 演算子を誤って多重定義しようとしてしまった際のエラー例（REPL）

```julia
julia> function (+)(p1::Point2D, p2::Point2D)
           Point2D(p1.x + p2.x, p1.y + p2.y)
       end
ERROR: error in method definition: function Base.+ must be explicitly imported to be extended
Stacktrace:
 [1] top-level scope
   @ none:0
 [2] top-level scope
   @ REPL[XX]:1
````

### 5-4-3. 型昇格ルール

#### コード5-26.	型昇格（`promote()`）の例

In [7]:
promote(1, 2)

(1, 2)

In [8]:
typeof((1, 2))

Tuple{Int64, Int64}

In [9]:
typeof(promote(1, 2))

Tuple{Int64, Int64}

In [10]:
promote(1, 3.4)

(1.0, 3.4)

In [11]:
typeof((1, 3.4))

Tuple{Int64, Float64}

In [12]:
typeof(promote(1, 3.4))

Tuple{Float64, Float64}

#### コード5-27.	`promote()` の利用を伴う関数（演算子）の多重定義例

In [13]:
⊓(x::Real, y::Real) = ⊓(promote(x, y)...)

⊓ (generic function with 1 method)

In [14]:
⊓(x::T, y::T) where {T <: Real} = max(x, y)

⊓ (generic function with 2 methods)

In [15]:
1 ⊓ 2

2

In [16]:
3.4 ⊓ 1.2

3.4

In [17]:
4 ⊓ π

4.0

In [18]:
3f0 ⊓ 4

4.0f0

In [19]:
3f0 ⊓ 4 ⊓ 2.0

4.0

#### 仮想コード5-3. `GeometricSequence` の定義例中における `promote()` の利用例（前節のコード5-20.より抜粋再掲）

```julia
function GeometricSequence(a::T1, r::T2, n::Integer) where {T1<:Number, T2<:Number}
    GeometricSequence(promote(a, r)..., Int(n))
end
```